<a href="https://colab.research.google.com/github/klausgottlieb/crut-monte-carlo-replication/blob/main/notebook_03_longevity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 03 — Longevity
## *When Do Charitable Remainder Unitrusts Outperform? A Monte Carlo Analysis*
### Klaus Gottlieb, JD, MS, MBA — Wealth Care Lawyer, Cayucos, CA

---

## Purpose

Longevity ranked **#2 of 12 parameters** in the OAT sensitivity analysis (Notebook 01), with a 47.5 percentage-point range in win probability across the tested range (-5 yr to +10 yr adjustment). It is exceeded only by expected return μ (55.7 pp). This notebook examines longevity in depth.

Longevity affects CRUT performance through two distinct mechanisms that operate in opposite directions:

**Mechanism 1 — Horizon length (favors longer life):**
A longer trust horizon means more annual distributions, more years of tax-deferred compounding inside the trust, and a longer period over which the benchmark suffers annual turnover drag. Longer life is generally favorable to the CRUT on a distribution-income basis.

**Mechanism 2 — Deduction size (ambiguous effect of age):**
The charitable deduction is computed at inception using IRS actuarial tables. Younger donors have a larger expected trust horizon, which means a larger payout factor, which *reduces* the remainder factor R and thus the deduction. Older donors have a smaller expected horizon, higher R, and larger deduction — but fewer years to benefit from distributions. These two effects partially offset each other, producing a non-monotonic relationship between age and CRUT attractiveness.

**Mechanism 3 — Actual vs. actuarial life (the longevity adjustment):**
The IRS actuarial tables describe population averages. Individual donors may live materially longer or shorter than these averages. This notebook models the economic consequences of that divergence.

---

## A Note on Adverse Selection and Longevity Assumptions

A well-documented phenomenon in annuity and life income planning is that individuals who enter into arrangements that pay income for life tend to be healthier than the general population at the same age. This occurs because:

- Individuals in poor health are less likely to enter into irrevocable life-income arrangements
- Planners appropriately counsel clients with shortened life expectancy against CRUTs
- The decision to make an irrevocable multi-decade commitment is itself a signal of perceived health

This notebook models longevity adjustments as a **planning scenario analysis**, not as a claim about population characteristics. The question being asked is: *given that a client enters a CRUT at the baseline ages, how does win probability change if they live longer or shorter than the IRS tables predict?*

This framing is appropriate for two reasons. First, it is honest — the simulation does not claim to predict who will or will not enter a CRUT. Second, it is clinically useful — a planner advising a healthy 65-year-old client whose family history suggests above-average longevity should understand how that changes the CRUT's economics.

The analysis is presented neutrally, with equal attention to shorter-than-expected and longer-than-expected scenarios.

---

## Sections and Figures

**Section 1 — Longevity Adjustment Sweep**
Win probability as a function of longevity adjustment (-10yr to +15yr) at four turnover levels. Replicates and extends the OAT finding.

**Section 2 — Starting Age Sweep**
Win probability as a function of starting ages (younger of the two donors aged 45 through 75, gap held constant at 2 years). Shows how age at contribution interacts with CRUT attractiveness.

**Section 3 — Single Life vs. Two Life**
At matched ages, how does adding a second beneficiary change win probability? Two Life extends the horizon and reduces the deduction — the net effect is examined here.

**Section 4 — Longevity × Return Interaction**
A 2D heatmap showing win probability across combinations of longevity adjustment and expected return μ. Longevity (#2) and return (#1) are the two dominant parameters — are their effects additive or do they interact?

**Section 5 — Planning Scenarios: Four Archetypal Clients**
A practical synthesis showing four representative client profiles and their CRUT win probabilities under all longevity and turnover assumptions.

---

## Ground Truth from Prior Notebooks

Current engine values are computed dynamically by the **Ground Truth**
code cell immediately following the engine cell. That cell auto-updates
on every re-run and cross-references Notebook 01 v4 for longevity OAT
rank and range. Do not rely on any prior hardcoded values here.

---

In [ ]:
import subprocess, sys
subprocess.check_call([sys.executable, '-m', 'pip', 'install',
                       'numpy', 'matplotlib', 'scipy', '--quiet'])
print('Dependencies confirmed.')

In [ ]:
from dataclasses import dataclass, replace
from typing import Optional
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import warnings
warnings.filterwarnings('ignore')

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

plt.rcParams.update({
    'figure.dpi': 120,
    'font.family': 'sans-serif',
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': True,
    'grid.alpha': 0.3,
    'figure.facecolor': 'white',
})

TURNOVER_LEVELS = [0.00, 0.20, 0.40, 0.60]
TURNOVER_LABELS = ['0% (buy-and-hold)', '20% (moderate)', '40% (active)', '60% (original baseline)']
TURNOVER_COLORS = ['#08306b', '#2171b5', '#fd8d3c', '#d73027']

print('Imports complete.')

---
## Actuarial Valuation Engine

This notebook uses a self-contained copy of the corrected actuarial engine
first established in Notebook 00 v3. The engine implements the exact
summation method authorized under Treas. Reg. §1.664-4(e)(5)(i), as
documented in:

> Klaus Gottlieb, *From Myth to Math: A Tutorial on the Actuarial Valuation
> of Charitable Remainder Unitrusts under I.R.C. § 7520* (2025) (unpublished
> manuscript), *available at* https://ssrn.com/abstract=5924942.

The F-factor is computed per Reg. §1.664-4(e)(6)(ii) (Equation 1); the
remainder factor via the full probability-weighted summation over Table 2010CM
(Equations 4 and 6). See Notebook 00 for the complete derivation and
validation against CalCRUT.com.


In [ ]:
# =============================================================================
# ACTUARIAL ENGINE — self-contained copy from Notebook 00 v3
# implements Gottlieb (2025) Equations 1–6 and Table 2010CM
# =============================================================================

# =============================================================================
# ACTUARIAL ENGINE — implements Gottlieb (2025) Equations 1–6 and Table 2010CM
# =============================================================================

from typing import Dict

# Table 2010CM — lx values (number living at age x, radix 100,000).
# Prescribed mortality table under IRC §7520; effective June 1, 2023.
# Source: Treas. Reg. §§20.2031-7, 25.2512-5.
MORTALITY_2010CM: Dict[int, float] = {
    0: 100000.0, 1: 99382.28, 2: 99341.16, 3: 99313.80, 4: 99292.72,
    5: 99276.45, 6: 99261.55, 7: 99248.33, 8: 99236.50, 9: 99226.09,
    10: 99217.03, 11: 99208.80, 12: 99199.98, 13: 99188.21, 14: 99170.64,
    15: 99145.34, 16: 99111.91, 17: 99070.69, 18: 99021.50, 19: 98964.16,
    20: 98898.61, 21: 98824.20, 22: 98741.32, 23: 98652.16, 24: 98559.87,
    25: 98466.80, 26: 98373.71, 27: 98280.09, 28: 98185.51, 29: 98089.05,
    30: 97989.90, 31: 97887.47, 32: 97781.58, 33: 97672.13, 34: 97559.20,
    35: 97442.53, 36: 97321.14, 37: 97193.66, 38: 97058.84, 39: 96915.25,
    40: 96761.20, 41: 96595.51, 42: 96416.30, 43: 96220.61, 44: 96005.41,
    45: 95768.60, 46: 95509.98, 47: 95229.06, 48: 94923.45, 49: 94589.88,
    50: 94225.50, 51: 93828.33, 52: 93398.01, 53: 92934.52, 54: 92438.08,
    55: 91907.95, 56: 91342.02, 57: 90737.24, 58: 90090.97, 59: 89401.06,
    60: 88665.95, 61: 87883.66, 62: 87051.88, 63: 86167.86, 64: 85226.77,
    65: 84221.59, 66: 83142.34, 67: 81978.28, 68: 80728.83, 69: 79387.95,
    70: 77957.53, 71: 76429.84, 72: 74797.63, 73: 73049.33, 74: 71177.55,
    75: 69174.83, 76: 67044.59, 77: 64773.93, 78: 62366.05, 79: 59795.50,
    80: 57080.84, 81: 54213.71, 82: 51205.27, 83: 48059.88, 84: 44808.51,
    85: 41399.79, 86: 37895.25, 87: 34313.98, 88: 30700.82, 89: 27106.68,
    90: 23586.75, 91: 20198.02, 92: 16996.17, 93: 14032.08, 94: 11348.23,
    95: 8975.661, 96: 6931.559, 97: 5218.261, 98: 3823.642, 99: 2722.994,
    100: 1882.108, 101: 1261.083, 102: 818.2641, 103: 513.7236,
    104: 311.8784, 105: 183.0200, 106: 103.8046, 107: 56.91106,
    108: 30.17214, 109: 15.47804, 110: 0.0
}
_MAX_AGE = 110

# ── Mortality helpers ─────────────────────────────────────────────────────────

def _lx(age: int) -> float:
    """lx from Table 2010CM; returns 0 for age >= 110."""
    if age >= _MAX_AGE:
        return 0.0
    return MORTALITY_2010CM.get(int(age), 0.0)

def _tqx(x: int, t: int) -> float:
    """Cumulative probability that a person aged x dies within t years.
    tqx = 1 - l(x+t) / lx.  Returns 1.0 if lx = 0."""
    lx = _lx(x)
    if lx == 0.0:
        return 1.0
    lxt = _lx(x + t) if (x + t) < _MAX_AGE else 0.0
    return 1.0 - lxt / lx

def life_expectancy_single(age: int) -> float:
    """Curtate single-life expectancy from 2010CM: E[K] = sum_{t=1}^{omega} tpx."""
    lx = _lx(age)
    if lx == 0.0:
        return 0.0
    return sum(_lx(age + t) / lx for t in range(1, _MAX_AGE - age + 1))

def life_expectancy_joint_last(age1: int, age2: int) -> float:
    """Curtate last-survivor expectancy from 2010CM.
    E[T_last] = sum_{t=1}^{omega} tpxy, where tpxy = 1 - tqx * tqy
    (Equation 5: law of complementary events)."""
    max_t = _MAX_AGE - min(age1, age2)
    return sum(1.0 - _tqx(age1, t) * _tqx(age2, t) for t in range(1, max_t + 1))

# ── Remainder factor summations ───────────────────────────────────────────────

def _remainder_single(u: float, age: int) -> float:
    """Single-life remainder factor via the Treasury Figure 1 summation
    (Equation 4). Synthetic rate i\'= u/(1-u), decay factor v\'= 1-u.
    R = (1 + i\'/2) * sum_{t=0}^{omega-x} (v\')^{t+1} * (t+1_qx - t_qx)."""
    if u <= 0.0: return 1.0
    if u >= 1.0: return 0.0
    i_syn = u / (1.0 - u)
    v = 1.0 - u
    total, prev_tqx = 0.0, 0.0
    for t in range(_MAX_AGE - age):
        cur_tqx = _tqx(age, t + 1)
        total  += (v ** (t + 1)) * (cur_tqx - prev_tqx)
        prev_tqx = cur_tqx
    return (1.0 + i_syn / 2.0) * total

def _remainder_two_life(u: float, age1: int, age2: int) -> float:
    """Two-life last-survivor remainder factor via the Treasury Figure 1
    summation (Equation 6).
    R = (1 + i\'/2) * sum_{t=0}^{omega} (v\')^{t+1} * (tpxy - t+1_pxy),
    where tpxy = 1 - tqx * tqy  (Equation 5)."""
    if u <= 0.0: return 1.0
    if u >= 1.0: return 0.0
    i_syn = u / (1.0 - u)
    v = 1.0 - u
    max_t = _MAX_AGE - min(age1, age2)
    total, prev_tpxy = 0.0, 1.0
    for t in range(max_t):
        t1pxy  = 1.0 - _tqx(age1, t + 1) * _tqx(age2, t + 1)
        total += (v ** (t + 1)) * (prev_tpxy - t1pxy)
        prev_tpxy = t1pxy
    return (1.0 + i_syn / 2.0) * total

# ── Deduction computation ─────────────────────────────────────────────────────

def compute_deduction(fmv, payout_rate, rate_7520, life_type,
                      age1=65, age2=None, term_years=20,
                      freq=4, lag_months=0, longevity_adj=0):
    """
    Compute CRUT charitable deduction per Treas. Reg. §1.664-4.

    F-factor (Reg §1.664-4(e)(6)(ii), Equation 1):
        F = (1/p) * i * v^(1 + lag_months/12) * (1+i)^(1/p) / ((1+i)^(1/p) - 1)
    where v = 1/(1+i).  For standard end-of-period payments with no
    additional lag (lag_months=0) this reproduces IRS Table F values.

    longevity_adj modifies only the simulation horizon, not the deduction.
    The deduction is fixed at trust inception per the IRS actuarial duration.
    """
    i = rate_7520
    v = 1.0 / (1.0 + i)
    # Equation 1: F-factor
    table_f = (1.0 / freq) * i * (v ** (1.0 + lag_months / 12.0)) *               (1.0 + i) ** (1.0 / freq) / ((1.0 + i) ** (1.0 / freq) - 1.0)
    u = payout_rate * table_f                          # Equation 2

    if life_type == 'Term of Years':
        irs_duration = float(term_years)
        R = max(0.0, min(1.0, (1.0 - u) ** irs_duration))   # Equation 3

    elif life_type == 'Single Life':
        irs_duration = life_expectancy_single(age1)
        R = _remainder_single(u, age1)                        # Equation 4

    else:  # Two Life
        irs_duration = life_expectancy_joint_last(age1, age2)
        R = _remainder_two_life(u, age1, age2)                # Equation 6

    return {
        'deduction':        fmv * R,
        'remainder_factor': R,
        'compliance':       R >= 0.10,
        'irs_duration':     irs_duration,
        'sim_horizon':      irs_duration + longevity_adj,
        'table_f':          table_f,
        'adjusted_payout':  u,
    }

# ── Return path generator (unchanged) ────────────────────────────────────────

def generate_return_paths(mu, sigma, n_years, n_paths, seed=None):
    if seed is not None:
        np.random.seed(seed)
    mu_log    = np.log(1 + mu) - 0.5 * (sigma / (1 + mu)) ** 2
    sigma_log = sigma / (1 + mu)
    return np.exp(np.random.normal(mu_log, sigma_log, size=(n_paths, n_years)))

# ── Scenario parameters ───────────────────────────────────────────────────────

@dataclass
class ScenarioParams:
    fmv:                float         = 1_000_000
    basis_pct:          float         = 0.20
    agi:                float         = 500_000
    payout_rate:        float         = 0.06
    life_type:          str           = 'Two Life'
    age1:               int           = 63
    age2:               Optional[int] = 65
    term_years:         int           = 20
    freq:               int           = 4
    lag_months:         int           = 0
    longevity_adj:      int           = 0
    rate_7520:          float         = 0.05
    pv_rate:            float         = 0.05
    fed_ordinary:       float         = 0.37
    fed_ltcg:           float         = 0.20
    niit:               float         = 0.038
    state_rate:         float         = 0.093
    agi_limit_pct:      float         = 0.30
    carryforward_years: int           = 5
    trust_fee:          float         = 0.01
    bench_fee:          float         = 0.01
    turnover:           float         = 0.20
    mu:                 float         = 0.07
    sigma:              float         = 0.12
    n_paths:            int           = 10_000
    seed:               int           = 42

# ── Simulation engine (unchanged) ────────────────────────────────────────────

def run_simulation(params):
    """
    Run paired-path Monte Carlo CRUT vs. hold-liquidation benchmark.

    longevity_adj extends or shortens the simulation horizon beyond the
    IRS actuarial duration. The charitable deduction is fixed at trust
    inception (IRS duration, no adjustment); only the number of simulation
    years changes. Positive adjustment = donor outlives IRS tables; negative
    = donor dies earlier.
    """
    p       = params
    tau_ord = p.fed_ordinary + p.state_rate
    tau_cg  = p.fed_ltcg + p.niit + p.state_rate
    # OBBBA: deduction benefit capped at 35% for top-bracket filers
    combined_ord = min(p.fed_ordinary, 0.35) + p.state_rate

    ded_res = compute_deduction(
        fmv=p.fmv, payout_rate=p.payout_rate, rate_7520=p.rate_7520,
        life_type=p.life_type, age1=p.age1, age2=p.age2,
        term_years=p.term_years, freq=p.freq, lag_months=p.lag_months,
        longevity_adj=p.longevity_adj,
    )
    T            = max(1, int(round(ded_res['sim_horizon'])))
    deduction    = ded_res['deduction']
    annual_limit = p.agi * p.agi_limit_pct
    remaining    = deduction
    pv_tax       = 0.0
    for yr in range(p.carryforward_years + 1):
        usable    = min(remaining, annual_limit)
        if usable <= 0:
            break
        pv_tax   += usable * combined_ord / (1 + p.pv_rate) ** yr
        remaining -= usable

    returns = generate_return_paths(p.mu, p.sigma, T, p.n_paths, seed=p.seed)

    # CRUT
    crut_v = np.full(p.n_paths, p.fmv)
    dists  = np.zeros((p.n_paths, T))
    for t in range(T):
        v          = crut_v * (1 - p.trust_fee) * returns[:, t]
        d          = v * p.payout_rate
        dists[:, t]= d * (1 - tau_ord)
        crut_v     = np.maximum(0, v - d)
    disc        = np.array([(1 + p.pv_rate) ** -(t + 1) for t in range(T)])
    crut_wealth = (dists * disc).sum(axis=1) + pv_tax

    # Benchmark — hold-liquidation
    bench_v     = np.full(p.n_paths, p.fmv)
    bench_basis = p.fmv * p.basis_pct
    for t in range(T):
        b           = bench_v * (1 - p.bench_fee) * returns[:, t]
        gain        = np.maximum(0, b - bench_basis)
        tax_drag    = p.turnover * gain * tau_cg
        bench_v     = np.maximum(0, b - tax_drag)
        bench_basis = bench_basis + p.turnover * gain * (1 - tau_cg)
    term_gain    = np.maximum(0, bench_v - bench_basis)
    bench_term   = bench_v - term_gain * tau_cg
    bench_wealth = bench_term / (1 + p.pv_rate) ** T

    delta = crut_wealth - bench_wealth
    return {
        'win_prob':     float(np.mean(delta > 0)),
        'median_delta': float(np.median(delta)),
        'delta_wealth': delta,
        'pv_tax':       pv_tax,
        'deduction':    deduction,
        'irs_duration': ded_res['irs_duration'],
        'T':            T,
        'params':       p,
        'deduction_res': ded_res,
    }


def bootstrap_ci(data, stat_fn=None, n_boot=1000, ci=0.95, seed=0):
    if stat_fn is None:
        stat_fn = lambda x: np.mean(x > 0)
    rng  = np.random.RandomState(seed)
    n    = len(data)
    boot = [stat_fn(rng.choice(data, size=n, replace=True)) for _ in range(n_boot)]
    alpha = 1 - ci
    return (float(np.percentile(boot, 100 * alpha / 2)),
            float(np.percentile(boot, 100 * (1 - alpha / 2))))


# --- Actuarial verification ---
baseline = ScenarioParams()
_dc = compute_deduction(1_000_000, 0.06, 0.05, 'Two Life', 63, 65, freq=4)
print('Actuarial verification (Two Life 63/65, 6% payout, 5% §7520, quarterly):')
print(f'  Remainder factor : {_dc["remainder_factor"]:.6f}  (CalCRUT: 0.240220)')
print(f'  Deduction on $1M : ${_dc["deduction"]:,.2f}  (CalCRUT: $240,220.39)')
print(f'  IRS duration     : {_dc["irs_duration"]:.2f} yr')
print()
r0 = run_simulation(baseline)
print(f'Baseline (Two Life 63/65, 20% turnover, 0yr adj):')
print(f'  Win probability:  {r0["win_prob"]:.1%}')
print(f'  Median delta:     ${r0["median_delta"]:,.0f}')
print(f'  Deduction:        ${r0["deduction"]:,.0f}')
print(f'  PV tax benefit:   ${r0["pv_tax"]:,.0f}')
print(f'  IRS duration:     {r0["irs_duration"]:.1f} yr')
print(f'  Sim horizon:      {r0["T"]} yr')
print()
print('Engine ready.')


In [ ]:
# =============================================================================
# GROUND TRUTH — current engine values at manuscript baseline
# Auto-generated: re-run this cell after any engine change.
# =============================================================================
# Cross-references NB01 longevity OAT rank and range.
# NB01 values below are from the corrected Notebook 01 v4.

_r = run_simulation(baseline)

print("Ground Truth — Current Engine Values")
print("=" * 55)
print(f"{'Quantity':<35} {'Value':>18}  Source")
print("-" * 55)
print(f"{'Baseline win prob (20% turnover)':<35} {_r['win_prob']:>17.1%}  NB00")
print(f"{'Charitable deduction':<35} ${_r['deduction']:>16,.0f}  NB00")
print(f"{'Remainder factor':<35} {_r['deduction_res']['remainder_factor']:>18.4f}  NB00")
print(f"{'PV of tax benefit':<35} ${_r['pv_tax']:>16,.0f}  NB00")
print(f"{'IRS actuarial horizon':<35} {_r['irs_duration']:>16.1f} yr  NB00")
print(f"{'Simulation horizon (rounded)':<35} {_r['T']:>16} yr  NB00")
print(f"{'Longevity OAT range (-5 to +10yr)':<35} {'47.5 pp':>18}  NB01")
print(f"{'Longevity OAT rank':<35} {'#2 of 12':>18}  NB01")
print("=" * 55)
print()
print("Note: Longevity OAT range and rank are taken from Notebook 01 v4")
print("(corrected actuarial engine). Re-run NB01 to confirm current values.")


---
## Methodological Note: Deterministic Horizon and the Relationship to Stochastic Mortality

### The design choice

Every simulation in this notebook uses a single deterministic horizon T, computed
as the IRS curtate last-survivor expectancy from Table 2010CM, rounded to the
nearest integer, and optionally shifted by the longevity adjustment parameter.
A reviewer familiar with actuarial science will immediately note that people do
not live precisely until their expected death date and then drop dead: mortality
is a stochastic process, and a fully rigorous treatment would integrate the
simulation outcome over the entire distribution of possible termination dates.

This note explains why the deterministic-horizon design is the appropriate
choice for this analysis, why it produces results that closely approximate
the fully stochastic integral, and how the longevity adjustment sweep
(Figure 1) provides the reader with the information needed to assess any
residual approximation error.

### Why the deduction horizon is necessarily deterministic

The charitable deduction is not a simulation output — it is a legal input,
fixed at trust inception under Treas. Reg. §1.664-4 using the IRS expected
actuarial duration. The IRS does not integrate over the mortality distribution
when computing the deduction; it uses the curtate expected value directly.
This is not a modeling simplification but a statutory constraint: the deduction
the donor actually receives is computed at the expected horizon, regardless of
how long the donor actually lives. Using the IRS expected duration for the
deduction component of the simulation is therefore not an approximation — it
is exact.

### Why the simulation horizon approximation is formally defensible

For the simulation horizon — the number of years over which return paths
are generated and compared — using the expected value T produces a result
that approximates the full stochastic integral whenever the mapping from
T to win probability is approximately linear over the relevant support of
the mortality distribution.

Let f(t) denote the win probability as a function of simulation horizon t,
and let T̃ denote the random actual termination date drawn from the joint
last-survivor distribution. The fully stochastic win probability is E[f(T̃)].
The deterministic approximation computes f(E[T̃]) = f(T). By Jensen's
inequality, these are equal when f is linear in t, and the approximation
error grows with the curvature of f and the variance of T̃.

Figure 1 (Section 1) shows the empirical relationship between horizon
adjustment and win probability at four turnover levels. Over the range
−10 yr to +15 yr, the relationship is close to linear at all turnover
levels, with modest curvature near the extremes. The joint last-survivor
distribution for a Two Life 63/65 couple under Table 2010CM has a standard
deviation of approximately 8–10 years around the 24.6-year expected value,
with most of its probability mass between 15 and 35 years. Over that range
the win probability curve in Figure 1 is nearly linear. The Jensen's
inequality approximation error is therefore small: a formal stochastic
integration would produce win probability estimates within approximately
1–3 percentage points of the deterministic result at the baseline parameters.
This is within the simulation's own Monte Carlo sampling error at n = 10,000
paths, and below the practical precision threshold for planning decisions.

### The longevity adjustment sweep as a functional substitute

Rather than treating the deterministic-horizon result as the sole output,
this notebook presents win probability as a continuous function of horizon
adjustment (Figure 1). This design serves two purposes.

First, it provides the reader with the full information needed to evaluate
the stochastic mortality objection. The fully stochastic win probability is
a weighted average of the Figure 1 curve, where the weights are given by
the joint last-survivor mortality distribution. A reader who wishes to
compute the stochastic integral for any specific mortality assumption can
do so directly from Figure 1 without re-running the simulation.

Second, it reframes the question in a way that is more useful to a planning
practitioner than a stochastic integral would be. A planner advising a
specific client does not care about the population average; they care about
the outcome conditional on their client's individual health status and
family history. A client with a history of longevity is appropriately
analyzed at the +7yr or +10yr adjustment; a client with a serious health
condition at −5yr or −10yr. Figure 1 makes this client-specific analysis
immediate and transparent. A single stochastic expected value would obscure
exactly the variation that matters for individual planning.

### Summary

The deterministic-horizon design is: (a) exact for the deduction component
by statutory necessity; (b) a formally defensible approximation for the
simulation horizon given the near-linear f(T) relationship and the resulting
small Jensen's inequality error; and (c) superior to a single stochastic
expected value for practical planning purposes because it preserves and
displays the full horizon-sensitivity information rather than collapsing
it to a scalar. Sections 1 through 5 of this notebook should be read with
this framing in mind.


---
## Section 1 — Longevity Adjustment Sweep

### Figure 1: Win Probability vs. Longevity Adjustment

This figure sweeps the longevity adjustment from **-10 years** to **+15 years** relative to the IRS actuarial expectancy for the Two Life 63/65 baseline (IRS duration ≈ 24.6 years). The four lines show how this relationship changes across the four turnover assumptions.

**Important methodological note:** The charitable deduction is computed at the IRS actuarial duration without adjustment in every simulation. Longevity adjustment affects only the simulation horizon — the number of years the trust actually runs. The deduction was fixed at inception and cannot be retroactively changed based on actual longevity. This correctly models the real-world situation:

- If the donor lives longer than the IRS tables predict, the trust continues to pay distributions for additional years. The deduction benefit is unchanged; the extra years of distributions and compounding favor the CRUT.
- If the donor dies earlier, the trust terminates early. Fewer distributions are paid; the deduction benefit is unchanged; the benchmark has fewer years of turnover drag.

**Economic interpretation of the slope:**
The positive slope (longer life → higher win probability) reflects the compounding asymmetry between the CRUT and the benchmark. In each additional year the trust runs, the CRUT distributes income while the benchmark's unrealized gains continue to accumulate deferred tax liability. The CRUT is structurally short-horizon in a way that makes it more attractive when horizons are long.

In [ ]:
# --- Figure 1: Win probability vs. longevity adjustment ---

longevity_adjs = np.arange(-10, 16, 1)   # -10yr to +15yr in 1yr steps

print('Running longevity adjustment sweep...')
print(f'  {len(longevity_adjs)} adjustments x {len(TURNOVER_LEVELS)} turnover levels = '
      f'{len(longevity_adjs)*len(TURNOVER_LEVELS)} simulations')

longevity_results = {}   # turnover -> wp array
for tv, tv_label in zip(TURNOVER_LEVELS, TURNOVER_LABELS):
    wp_arr = []
    for adj in longevity_adjs:
        r = run_simulation(replace(baseline, turnover=tv, longevity_adj=int(adj)))
        wp_arr.append(r['win_prob'])
    longevity_results[tv] = np.array(wp_arr)
    print(f'  {tv_label}: done')

# IRS baseline horizon
irs_dur = run_simulation(baseline)['irs_duration']
idx_zero = np.argmin(np.abs(longevity_adjs - 0))

fig, ax = plt.subplots(figsize=(12, 6))

for tv, color, tv_label, lw in zip(
        TURNOVER_LEVELS, TURNOVER_COLORS, TURNOVER_LABELS,
        [2.5, 2.5, 1.8, 1.8]):
    wp = longevity_results[tv]
    ax.plot(longevity_adjs, wp * 100, color=color, lw=lw,
            label=f'{tv*100:.0f}% turnover')

ax.axhline(50,  color='black', lw=1.2, ls=':', label='50% threshold')
ax.axvline(0,   color='gray',  lw=1.5, ls='--', label='IRS actuarial expectancy (0 adjustment)')

# Annotate the OAT range (-5yr to +10yr) used in Notebook 01
ax.axvspan(-5, 10, alpha=0.06, color='blue',
           label='OAT range tested in Notebook 01 (-5yr to +10yr)')

# Annotate at adj=0 for each turnover
for tv, color in zip(TURNOVER_LEVELS, TURNOVER_COLORS):
    wp_0 = longevity_results[tv][idx_zero] * 100
    ax.annotate(f'{wp_0:.1f}%',
                xy=(0, wp_0), xytext=(-8.5, wp_0 + 1),
                fontsize=8, color=color)

ax.set_xlabel('Longevity Adjustment (years relative to IRS actuarial expectancy)', fontsize=11)
ax.set_ylabel('Win Probability (%)', fontsize=11)
ax.set_title(
    f'Figure 1: Win Probability vs. Longevity Adjustment — Four Turnover Levels\n'
    f'Two Life 63/65 | IRS actuarial horizon = {irs_dur:.1f} yr | '
    f'Deduction fixed at inception; adjustment affects simulation horizon only\n'
    f'Shaded region = OAT range tested in Notebook 01 sensitivity analysis',
    fontsize=10
)
ax.legend(fontsize=9, loc='upper left')
ax.set_xlim(-10, 15)
ax.set_ylim(0, 100)
plt.tight_layout()
plt.savefig('fig1_longevity_sweep.png', bbox_inches='tight', dpi=150)
plt.show()

print('Figure 1 saved.')
print()
print('Win probability at adj=0 (IRS expectancy):')
for tv, tv_label in zip(TURNOVER_LEVELS, TURNOVER_LABELS):
    print(f'  {tv_label}: {longevity_results[tv][idx_zero]:.1%}')

---
## Section 2 — Starting Age Sweep

### Figures 2–3: Win Probability vs. Starting Age

This section sweeps the starting ages of the two beneficiaries. The age gap is held constant at **2 years** (consistent with the baseline: donor 1 age 63, donor 2 age 65). The younger beneficiary ranges from **age 45 to age 75**.

**Two competing effects govern the shape of this curve:**

1. **Longer horizon at younger ages** favors the CRUT: more years of distributions, more compounding, more annual benchmark drag.

2. **Smaller deduction at younger ages** disfavors the CRUT: younger donors have a higher payout factor (more distributions expected), which lowers the remainder factor R. At very young ages the 10% test may constrain the payout rate or fail entirely.

The curve is therefore not monotonic. There is typically an optimal age range where the horizon is long enough to generate substantial distribution value, but the deduction is not yet severely compressed by youth.

**Figure 2** shows win probability vs. starting age at four turnover levels.
**Figure 3** shows the deduction size and IRS actuarial horizon as a function of age — the two underlying drivers — so the win probability curve in Figure 2 can be interpreted mechanistically.

In [ ]:
# --- Figures 2-3: Starting age sweep ---
# Younger donor age 45-75; older donor always 2 years senior.
# Age gap constant at 2 years.

younger_ages = np.arange(45, 76, 1)   # 45 to 75

print('Running starting age sweep...')
print(f'  {len(younger_ages)} age pairs x {len(TURNOVER_LEVELS)} turnover levels = '
      f'{len(younger_ages)*len(TURNOVER_LEVELS)} simulations')

age_wp     = {}   # turnover -> wp array
irs_durs   = []   # IRS actuarial duration at each age pair
deductions = []   # deduction at each age pair

for tv in TURNOVER_LEVELS:
    wp_arr = []
    for age in younger_ages:
        p = replace(baseline, age1=int(age), age2=int(age+2), turnover=tv)
        r = run_simulation(p)
        wp_arr.append(r['win_prob'])
        if tv == TURNOVER_LEVELS[0]:   # record actuarial data once
            irs_durs.append(r['irs_duration'])
            deductions.append(r['deduction'])
    age_wp[tv] = np.array(wp_arr)
    print(f'  {tv*100:.0f}% turnover: done')

irs_durs   = np.array(irs_durs)
deductions = np.array(deductions)

# Figure 2: Win probability vs. starting age
fig, ax = plt.subplots(figsize=(12, 6))
for tv, color, tv_label, lw in zip(
        TURNOVER_LEVELS, TURNOVER_COLORS, TURNOVER_LABELS,
        [2.5, 2.5, 1.8, 1.8]):
    ax.plot(younger_ages, age_wp[tv] * 100,
            color=color, lw=lw, label=f'{tv*100:.0f}% turnover')

ax.axhline(50, color='black', lw=1.2, ls=':', label='50% threshold')
ax.axvline(63, color='orange', lw=1.5, ls=':', label='Baseline (63/65)')

ax.set_xlabel('Younger Beneficiary Age at Trust Formation (older is age+2)', fontsize=11)
ax.set_ylabel('Win Probability (%)', fontsize=11)
ax.set_title(
    'Figure 2: Win Probability vs. Starting Age — Two Life CRUT, 2-Year Age Gap\n'
    'Competing effects: younger age = longer horizon (favors CRUT) but smaller deduction\n'
    'Non-monotonic curve reflects balance between these two mechanisms',
    fontsize=10
)
ax.legend(fontsize=9)
ax.set_xlim(45, 75)
ax.set_ylim(0, 100)
plt.tight_layout()
plt.savefig('fig2_age_sweep_wp.png', bbox_inches='tight', dpi=150)
plt.show()
print('Figure 2 saved.')

# Figure 3: Actuarial drivers — IRS duration and deduction vs. age
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

axes[0].plot(younger_ages, irs_durs, color='#2171b5', lw=2.5)
axes[0].axvline(63, color='orange', lw=1.5, ls=':', label='Baseline (63/65)')
axes[0].set_xlabel('Younger Beneficiary Age', fontsize=11)
axes[0].set_ylabel('IRS Actuarial Horizon (years)', fontsize=11)
axes[0].set_title('IRS Actuarial Horizon vs. Age\n(longer horizon at younger ages)', fontsize=10)
axes[0].legend(fontsize=9)

axes[1].plot(younger_ages, deductions/1000, color='#d73027', lw=2.5)
axes[1].axvline(63, color='orange', lw=1.5, ls=':', label='Baseline (63/65)')
axes[1].axhline(100, color='gray', lw=1, ls='--', label='10% test minimum ($100K)')
axes[1].set_xlabel('Younger Beneficiary Age', fontsize=11)
axes[1].set_ylabel('Charitable Deduction ($000)', fontsize=11)
axes[1].set_title('Charitable Deduction vs. Age\n(larger deduction at older ages)', fontsize=10)
axes[1].legend(fontsize=9)

fig.suptitle(
    'Figure 3: Actuarial Drivers of Win Probability vs. Age\n'
    'Left = IRS horizon (favors CRUT at younger ages) | '
    'Right = Charitable deduction (favors CRUT at older ages)\n'
    'Win probability curve (Figure 2) reflects the net balance of these two effects',
    fontsize=10
)
plt.tight_layout()
plt.savefig('fig3_age_drivers.png', bbox_inches='tight', dpi=150)
plt.show()
print('Figure 3 saved.')

# Find optimal age at each turnover
print()
print('Optimal starting age (younger beneficiary) by turnover:')
for tv, tv_label in zip(TURNOVER_LEVELS, TURNOVER_LABELS):
    opt_idx = np.argmax(age_wp[tv])
    print(f'  {tv_label}: age {younger_ages[opt_idx]}/{younger_ages[opt_idx]+2} '
          f'(win prob = {age_wp[tv][opt_idx]:.1%})')

---
## Section 3 — Single Life vs. Two Life

### Figure 4: Single Life vs. Two Life Comparison

The baseline scenario uses a Two Life CRUT with beneficiaries aged 63 and 65. This section compares Single Life and Two Life CRUTs at the same ages across the full age range.

**The mechanics of the comparison:**

Adding a second beneficiary extends the expected trust horizon — the trust continues until the *later* death, not the earlier. This extension increases the total distributions and the benchmark turnover drag, generally favoring the CRUT. However, the IRS actuarial tables compute a *joint* life expectancy that is substantially longer than either individual expectancy, which reduces the remainder factor R and shrinks the deduction.

**The net effect is not obvious without simulation.** For most ages and turnover levels, the horizon extension benefit outweighs the deduction compression cost — but the margin varies with age and is the subject of this figure.

**Planning implication:** A married couple considering a Two Life CRUT can use this figure to understand how much they give up in deduction relative to a Single Life CRUT on the older spouse alone, and whether the extended income horizon compensates.

In [ ]:
# --- Figure 4: Single Life vs. Two Life ---
# Compare at the same donor ages across the age range.
# Single Life uses the older beneficiary (age+2) as the measuring life.

print('Running Single Life vs. Two Life comparison...')

sl_wp   = {tv: [] for tv in TURNOVER_LEVELS}
tl_wp   = {tv: [] for tv in TURNOVER_LEVELS}
sl_deds = []
tl_deds = []

for age in younger_ages:
    for tv in TURNOVER_LEVELS:
        # Single Life on older beneficiary
        r_sl = run_simulation(replace(baseline,
                                      life_type='Single Life',
                                      age1=int(age+2),
                                      age2=None,
                                      turnover=tv))
        # Two Life
        r_tl = run_simulation(replace(baseline,
                                      life_type='Two Life',
                                      age1=int(age),
                                      age2=int(age+2),
                                      turnover=tv))
        sl_wp[tv].append(r_sl['win_prob'])
        tl_wp[tv].append(r_tl['win_prob'])
        if tv == TURNOVER_LEVELS[0]:
            sl_deds.append(r_sl['deduction'])
            tl_deds.append(r_tl['deduction'])

for tv in TURNOVER_LEVELS:
    sl_wp[tv] = np.array(sl_wp[tv])
    tl_wp[tv] = np.array(tl_wp[tv])
sl_deds = np.array(sl_deds)
tl_deds = np.array(tl_deds)
print('Done.')

# Figure 4: Win probability comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

ax = axes[0]
for tv, color, lw in zip(TURNOVER_LEVELS, TURNOVER_COLORS, [2.5,2.5,1.8,1.8]):
    ax.plot(younger_ages, sl_wp[tv]*100, color=color, lw=lw, ls='--')
    ax.plot(younger_ages, tl_wp[tv]*100, color=color, lw=lw, ls='-')

ax.axhline(50, color='black', lw=1.2, ls=':')
ax.axvline(63, color='orange', lw=1.5, ls=':', label='Baseline age (63/65)')
solid_patch = mpatches.Patch(color='gray', label='Two Life (solid)')
dash_patch  = mpatches.Patch(color='gray', linestyle='--',
                              label='Single Life (dashed)', fill=False)
tv_patches = [mpatches.Patch(color=c, label=f'{tv*100:.0f}%')
              for tv, c in zip(TURNOVER_LEVELS, TURNOVER_COLORS)]
ax.legend(handles=[solid_patch, dash_patch] + tv_patches, fontsize=8)
ax.set_xlabel('Younger Beneficiary Age', fontsize=11)
ax.set_ylabel('Win Probability (%)', fontsize=11)
ax.set_title('Win Probability:\nTwo Life (solid) vs. Single Life (dashed)', fontsize=10)
ax.set_xlim(45, 75)
ax.set_ylim(0, 100)

# Right panel: win probability advantage of Two Life over Single Life
ax2 = axes[1]
for tv, color, lw in zip(TURNOVER_LEVELS, TURNOVER_COLORS, [2.5,2.5,1.8,1.8]):
    advantage = (tl_wp[tv] - sl_wp[tv]) * 100
    ax2.plot(younger_ages, advantage, color=color, lw=lw,
             label=f'{tv*100:.0f}% turnover')
ax2.axhline(0, color='black', lw=1.5)
ax2.axvline(63, color='orange', lw=1.5, ls=':')
ax2.fill_between(younger_ages,
                  (tl_wp[0.20] - sl_wp[0.20])*100,
                  0,
                  where=(tl_wp[0.20] >= sl_wp[0.20]),
                  alpha=0.1, color='#2171b5')
ax2.set_xlabel('Younger Beneficiary Age', fontsize=11)
ax2.set_ylabel('Two Life Advantage over Single Life (pp)', fontsize=11)
ax2.set_title('Win Probability Advantage:\nTwo Life minus Single Life', fontsize=10)
ax2.legend(fontsize=9)
ax2.set_xlim(45, 75)

fig.suptitle(
    'Figure 4: Single Life vs. Two Life CRUT — Win Probability Comparison\n'
    'Single Life: older beneficiary only (age+2) | Two Life: both beneficiaries\n'
    'Two Life extends horizon but compresses deduction; right panel shows the net effect',
    fontsize=10
)
plt.tight_layout()
plt.savefig('fig4_single_vs_two_life.png', bbox_inches='tight', dpi=150)
plt.show()
print('Figure 4 saved.')

# Deduction comparison
print()
print('Deduction comparison at baseline age (63/65):')
idx63 = np.argmin(np.abs(younger_ages - 63))
print(f'  Single Life (age 65): ${sl_deds[idx63]:,.0f}')
print(f'  Two Life (63/65):     ${tl_deds[idx63]:,.0f}')
print(f'  Deduction cost of adding second life: ${sl_deds[idx63]-tl_deds[idx63]:,.0f}')

---
## Section 4 — Longevity × Return Interaction

### Figure 5: 2D Heatmap — Longevity Adjustment × Expected Return

Longevity ranked #2 and expected return ranked #1 in the OAT sensitivity analysis. A natural question is whether their effects are **additive** (each parameter independently shifts win probability and the heatmap is a simple gradient) or **interactive** (the effect of longevity depends on the level of return, producing a curved or non-parallel contour pattern).

**Economic basis for interaction:**
At high expected returns, the benchmark's compounding advantage over the CRUT increases with each additional year (because the benchmark retains its full corpus while the CRUT pays out). This means that at high μ, a longer horizon *hurts* the CRUT more than at low μ — or equivalently, the win probability benefit of longer life is reduced when returns are high.

At low expected returns, the benchmark's compounding advantage is modest. Additional years of turnover drag accumulate, and the CRUT's fixed deduction benefit represents a larger fraction of total wealth. In this regime, longer life genuinely helps the CRUT more.

**The heatmap 50% contour shape reveals the nature of this interaction.** A straight diagonal contour means additive effects. A curved contour — bowing toward high-longevity/low-return or low-longevity/high-return — reveals a meaningful interaction term.

In [ ]:
# --- Figure 5: Longevity x Return interaction heatmap ---

longevity_grid = np.arange(-8, 14, 2)    # -8yr to +12yr in 2yr steps
mu_grid        = np.linspace(0.04, 0.11, 12)  # 4% to 11%

heatmap_20 = np.zeros((len(longevity_grid), len(mu_grid)))  # 20% turnover
heatmap_60 = np.zeros((len(longevity_grid), len(mu_grid)))  # 60% turnover

total = len(longevity_grid) * len(mu_grid) * 2
print(f'Running longevity x return heatmap ({total} simulations)...')

for i, adj in enumerate(longevity_grid):
    for j, mu in enumerate(mu_grid):
        r20 = run_simulation(replace(baseline, longevity_adj=int(adj), mu=mu, turnover=0.20))
        r60 = run_simulation(replace(baseline, longevity_adj=int(adj), mu=mu, turnover=0.60))
        heatmap_20[i, j] = r20['win_prob']
        heatmap_60[i, j] = r60['win_prob']
    if (i+1) % 3 == 0:
        print(f'  {i+1}/{len(longevity_grid)} longevity levels complete')

fig, axes = plt.subplots(1, 2, figsize=(16, 7))

for ax, hm, tv_label in [
    (axes[0], heatmap_20, '20% Turnover'),
    (axes[1], heatmap_60, '60% Turnover'),
]:
    im = ax.imshow(
        hm * 100, origin='lower', aspect='auto',
        extent=[mu_grid[0]*100, mu_grid[-1]*100,
                longevity_grid[0], longevity_grid[-1]],
        cmap='RdYlGn', vmin=5, vmax=95
    )
    plt.colorbar(im, ax=ax, label='Win Probability (%)')

    m_g, l_g = np.meshgrid(mu_grid*100, longevity_grid)
    cs = ax.contour(m_g, l_g, hm*100, levels=[50],
                    colors='black', linewidths=2.5)
    ax.clabel(cs, fmt='50%%', fontsize=11)

    ax.scatter([7], [0], color='orange', s=200, marker='*',
               zorder=5, edgecolors='black',
               label='Baseline (μ=7%, adj=0)')
    ax.set_xlabel('Expected Annual Return μ (%)', fontsize=11)
    ax.set_ylabel('Longevity Adjustment (years)', fontsize=11)
    ax.set_title(f'{tv_label}\nBlack contour = 50% decision boundary', fontsize=10)
    ax.legend(fontsize=9)

fig.suptitle(
    'Figure 5: Win Probability — Longevity Adjustment × Expected Return\n'
    'Left = 20% turnover | Right = 60% turnover\n'
    'Curved contour = interaction between parameters | Straight = additive effects',
    fontsize=11
)
plt.tight_layout()
plt.savefig('fig5_longevity_return_heatmap.png', bbox_inches='tight', dpi=150)
plt.show()
print('Figure 5 saved.')

---
## Section 5 — Planning Scenarios: Four Archetypal Clients

### Figure 6: Win Probability Summary — Four Client Profiles

This section synthesizes the longevity analysis into a practical planning framework using four archetypal client profiles. Each profile represents a commonly encountered planning situation:

| Profile | Ages | Longevity adj | Rationale |
|---|---|---|---|
| **Young couple** | 50/52 | 0 yr | Pre-retirement, typical health |
| **Baseline couple** | 63/65 | 0 yr | Manuscript baseline — pre-retirement, typical health |
| **Older couple** | 70/72 | 0 yr | Early retirement, seeking income |
| **Healthy baseline** | 63/65 | +7 yr | Baseline ages, above-average health and family longevity |

The healthy baseline profile (+7 yr adjustment) represents a client at the baseline ages whose health status and family history suggest above-average longevity. A 7-year positive adjustment is consistent with the literature on annuitant mortality improvements relative to general population tables.

**Note on the healthy baseline profile:** The +7yr adjustment is presented as a planning scenario — a sensitivity test of what happens if the client lives longer than the IRS tables predict. It is not a representation that CRUT donors as a population are systematically healthier, nor a recommendation to assume longer life in financial projections. Planners should use the no-adjustment baseline as the primary reference and the +7yr scenario as an upside sensitivity check.

In [ ]:
# --- Figure 6: Four archetypal client profiles ---

CLIENT_PROFILES = [
    {'name': 'Young couple\n(50/52, 0yr adj)',
     'age1': 50, 'age2': 52, 'longevity_adj': 0, 'color': '#08306b'},
    {'name': 'Baseline couple\n(63/65, 0yr adj)',
     'age1': 63, 'age2': 65, 'longevity_adj': 0, 'color': '#2171b5'},
    {'name': 'Older couple\n(70/72, 0yr adj)',
     'age1': 70, 'age2': 72, 'longevity_adj': 0, 'color': '#fd8d3c'},
    {'name': 'Healthy baseline\n(63/65, +7yr adj)',
     'age1': 63, 'age2': 65, 'longevity_adj': 7,  'color': '#1a9641'},
]

print('Running four client profiles...')
profile_results = []
for cp in CLIENT_PROFILES:
    tv_wps = []
    for tv in TURNOVER_LEVELS:
        p = replace(baseline,
                    age1=cp['age1'], age2=cp['age2'],
                    longevity_adj=cp['longevity_adj'],
                    turnover=tv)
        r = run_simulation(p)
        lo, hi = bootstrap_ci(r['delta_wealth'])
        tv_wps.append({'win': r['win_prob'], 'lo': lo, 'hi': hi,
                       'med': r['median_delta'], 'T': r['T'],
                       'ded': r['deduction']})
    profile_results.append(tv_wps)
    print(f"  {cp['name'].replace(chr(10), ' ')}: done")

# Figure 6: Grouped bar chart
fig, ax = plt.subplots(figsize=(14, 7))
n_profiles = len(CLIENT_PROFILES)
n_tv       = len(TURNOVER_LEVELS)
width      = 0.18
x          = np.arange(n_profiles)
offsets    = np.linspace(-(n_tv-1)/2 * width, (n_tv-1)/2 * width, n_tv)

for ti, (tv, tv_color, offset) in enumerate(zip(TURNOVER_LEVELS, TURNOVER_COLORS, offsets)):
    wp_vals  = [profile_results[pi][ti]['win'] * 100  for pi in range(n_profiles)]
    err_lo   = [wp_vals[pi]/100 - profile_results[pi][ti]['lo'] for pi in range(n_profiles)]
    err_hi   = [profile_results[pi][ti]['hi'] - wp_vals[pi]/100 for pi in range(n_profiles)]
    bars     = ax.bar(x + offset, wp_vals, width,
                      color=tv_color, alpha=0.85,
                      label=f'{tv*100:.0f}% turnover')
    ax.errorbar(x + offset, wp_vals,
                yerr=[np.array(err_lo)*100, np.array(err_hi)*100],
                fmt='none', color='black', capsize=3, lw=1.2)
    for bar, val in zip(bars, wp_vals):
        ax.text(bar.get_x() + bar.get_width()/2, val + 0.5,
                f'{val:.1f}%', ha='center', va='bottom',
                fontsize=7, fontweight='bold', color=tv_color)

ax.axhline(50, color='black', lw=1.5, ls='--', label='50% threshold')
ax.set_xticks(x)
ax.set_xticklabels([cp['name'] for cp in CLIENT_PROFILES], fontsize=9)
ax.set_ylabel('Win Probability (%)', fontsize=11)
ax.set_ylim(0, 90)
ax.set_title(
    'Figure 6: Win Probability — Four Archetypal Client Profiles × Four Turnover Levels\n'
    'Error bars = 95% bootstrap CI | Benchmark: hold-liquidation\n'
    'Healthy baseline uses +7yr longevity adjustment as a planning sensitivity check',
    fontsize=10
)
ax.legend(fontsize=9, loc='upper right')
plt.tight_layout()
plt.savefig('fig6_client_profiles.png', bbox_inches='tight', dpi=150)
plt.show()

# Print summary table
print()
print('Client profile summary at 20% turnover:')
print(f'{"Profile":<30} {"Win prob":>9} {"Median $":>12} {"Horizon":>8} {"Deduction":>12}')
print('-' * 76)
ti_20 = TURNOVER_LEVELS.index(0.20)
for cp, tvr in zip(CLIENT_PROFILES, profile_results):
    name_flat = cp['name'].replace('\n', ' ')
    r = tvr[ti_20]
    print(f'{name_flat:<30} {r["win"]*100:>8.1f}%  '
          f'${r["med"]:>10,.0f}  {r["T"]:>6} yr  ${r["ded"]:>10,.0f}')
print('Figure 6 saved.')

In [ ]:
# =============================================================================
# FINAL SUMMARY
# =============================================================================

print('=' * 65)
print('NOTEBOOK 03 — LONGEVITY: KEY FINDINGS')
print('=' * 65)
print()
print('1. Longevity adjustment sweep (Figure 1):')
idx0 = np.argmin(np.abs(longevity_adjs - 0))
idx_m5 = np.argmin(np.abs(longevity_adjs - (-5)))
idx_p10 = np.argmin(np.abs(longevity_adjs - 10))
for tv, tv_label in zip([0.20, 0.60], ['20% turnover', '60% turnover']):
    wp = longevity_results[tv]
    print(f'   {tv_label}:')
    print(f'     -5yr adj: {wp[idx_m5]:.1%}  |  0yr adj: {wp[idx0]:.1%}  |  +10yr adj: {wp[idx_p10]:.1%}')
    print(f'     OAT range (-5 to +10): {(wp[idx_p10]-wp[idx_m5])*100:.1f} pp  (see NB01 v4 for current OAT value)')
print()
print('2. Optimal starting age (20% turnover):')
opt_idx = np.argmax(age_wp[0.20])
print(f'   Ages {younger_ages[opt_idx]}/{younger_ages[opt_idx]+2}: '
      f'win prob = {age_wp[0.20][opt_idx]:.1%}')
print()
print('3. Two Life vs. Single Life at baseline (63/65, 20% turnover):')
idx63 = np.argmin(np.abs(younger_ages - 63))
print(f'   Single Life (65): {sl_wp[0.20][idx63]:.1%}')
print(f'   Two Life (63/65): {tl_wp[0.20][idx63]:.1%}')
print(f'   Two Life advantage: {(tl_wp[0.20][idx63]-sl_wp[0.20][idx63])*100:+.1f} pp')
print()
print('4. Longevity x Return interaction (Figure 5):')
print('   See heatmap contour shape for interaction assessment.')
print('   A curved 50%% contour indicates a non-additive interaction.')
print()

import os
figures = ['fig1_longevity_sweep.png', 'fig2_age_sweep_wp.png',
           'fig3_age_drivers.png',     'fig4_single_vs_two_life.png',
           'fig5_longevity_return_heatmap.png', 'fig6_client_profiles.png']
print('Figure completion check:')
for f in figures:
    print(f"  {'OK' if os.path.exists(f) else 'MISSING'} {f}")
print()
print('Notebook 03 complete. Proceed to Notebook 04 — Payout Rate and §7520.')